In [258]:
import pandas as pd
from itertools import permutations
import requests
from bs4 import BeautifulSoup
import json
import polars as pl
# import geopy.distance
# from shapely.geometry import Point
# import geopandas as gpd
# from geopandas import GeoDataFrame
# import fiona
# from currency_converter import CurrencyConverter
# from bokeh.plotting import figure, save, show
# from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
# from pathlib import Path

In [259]:
#converter = CurrencyConverter()
#DESIRED_CUR = 'USD'

In [260]:
base_url = 'https://www.rome2rio.com/map/'
# airports_file_path = "../files/csv/Locations with airports.csv"

In [261]:
def get_url(url, english=True):
    if english:
        r = requests.get(url, headers={'Accept-Language': 'en-US,en;q=0.5'})
    else:
        r = requests.get(url)

    if r.status_code != 200:
        print("Invalid page!")
        return []
    else:
        data = {}
        soup = BeautifulSoup(r.content, 'html.parser')
        # data["title"] = soup.find('h1').text
        dis = soup.find('meta', {'id': 'deeplinkTrip'})
        parsed = json.loads(dis["content"])[2][1]

        # print(json.dumps(parsed, indent=4, sort_keys=True))
        return parsed


class Cities:
    def __init__(self):
        #self.path = cities_file
        self.base_url = 'https://www.rome2rio.com/map/'
        try:
            #self.df = pd.read_csv(filename)
            pass
        except FileNotFoundError:
            print("File Not Found")
        self.routes = {}
        # self.pairs = []

    """ def gen_combinations(self, n=2):
        self.pairs = permutations(self.df['city'], n) """
        
    """ def get_routes(self, city1, city2):
        pass """

    def scrap_routes(self, city1, city2):
        tmp_url = self.base_url + city1 + '/' + city2
        self.routes[(city1, city2)] = get_url(tmp_url)


In [262]:
from_city, to_city = 'Berlin', 'Montevideo'

In [263]:
# attempt to extract elementary routes for each path
c = Cities()
c.scrap_routes(from_city, to_city)
pathes = c.routes[(from_city, to_city)]

In [264]:
class BoundedBoxes():
    def __init__(self, path: str) -> None:
        self.df = pl.read_csv(path, has_header=False, new_columns=['id', 'lat_1', 'lat_2', 'lon_1', 'lon_2'])

    def get_bb_id(self, coords: list[float, float]) -> int:
        try:
            cond_1 = (coords[0] >= self.df['lat_1']) & (coords[0] <= self.df['lat_2'])
            cond_2 = (coords[1] >= self.df['lon_1']) & (coords[1] <= self.df['lon_2'])
        
            filter_df = self.df.filter(cond_1 & cond_2)
            
            return filter_df['id'][0]
        except:
            return -1       

In [265]:
class Airports():
    def __init__(self, path: str) -> None:
        self.df = pl.read_csv(path, has_header=False, new_columns=['code', 'id'])

    def get_airport_id(self, code: str):
        try:
            filter_df = self.df.filter(code.lower() == self.df['code']) 
            return filter_df['id'][0]
        except:
            return -1       

In [266]:
""" 1-level - avaliable pathes
    2-level  - path summary - path[8] 
    
    TRAIN, SUBWAY, BUS, BUSFERRY, CAR, CARFERRY, CARTRAIN, FERRY, RIDESHARE, TRAM
        *   estimate_price[] = [i][8][j][13]
        *   estimate_price[] LOCAL =  [i][8][j][14]
        *   duration, sec = [i][8][j][3]
        *   vehicle_type = [i][8][j][1]
        *   distance, km = [i][8][j][5]
        *   start_point[] = [i][8][j][6]
        *   end_point[] = [i][8][j][7]
        *   all_points[] = [i][8][j][18]
    FLY
        *   estimate_price[] = [i][8][j][11]
        *   estimate_price[] Local = ''
        *   duration, sec = [i][8][j][4]
        *   vehicle_type [flight] = [i][8][j][0]
        *   distance = [i][8][j][]
        *   start_point[] = [i][8][j][2]
        *   end_point[] = [i][8][j][3]
        *   all_points[] = [i][8][j][18] 
        *   aiport code = [i][8][j][2] 
"""
        
##for i, item in enumerate(pathes[0][8][0]):
    ##print(i, '\n\t', item)

" 1-level - avaliable pathes\n    2-level  - path summary - path[8] \n    \n    TRAIN, SUBWAY, BUS, BUSFERRY, CAR, CARFERRY, CARTRAIN, FERRY, RIDESHARE, TRAM\n        *   estimate_price[] = [i][8][j][13]\n        *   estimate_price[] LOCAL =  [i][8][j][14]\n        *   duration, sec = [i][8][j][3]\n        *   vehicle_type = [i][8][j][1]\n        *   distance, km = [i][8][j][5]\n        *   start_point[] = [i][8][j][6]\n        *   end_point[] = [i][8][j][7]\n        *   all_points[] = [i][8][j][18]\n    FLY\n        *   estimate_price[] = [i][8][j][11]\n        *   estimate_price[] Local = ''\n        *   duration, sec = [i][8][j][4]\n        *   vehicle_type [flight] = [i][8][j][0]\n        *   distance = [i][8][j][]\n        *   start_point[] = [i][8][j][2]\n        *   end_point[] = [i][8][j][3]\n        *   all_points[] = [i][8][j][18] \n        *   aiport code = [i][8][j][2] \n"

In [267]:
path_bboxes = "../files/csv/bbox short.csv"
path_airports = "../files/csv/airport codes short.csv"

output_path = "../files/output/csv_output/"
output_file = output_path + f'{from_city}-{to_city}' + '.csv'

bb = BoundedBoxes(path_bboxes)
ap = Airports(path_airports)

data = {'path_id':[],
        'path_name':[],
        'from_node':[],
        'to_node':[], 
        'from_id':[], 
        'to_id':[], 
        'transport':[], 
        'from_airport':[], 
        'to_airport':[],
        'from_airport_id':[], 
        'to_airport_id':[],
        'price':[], 'currency':[], 
        'price_local':[], 'currency_local':[],
        'distance_km':[], 
        'duration_min':[]}

for path_num, path in enumerate(pathes):
    for route_num, route in enumerate(path[8][:-1]):
        if route[0] in ('walk', 'hotel'):
            continue
        if route[0] == 'flight':
            data['path_id'].append(path_num)
            data['path_name'].append(path[4])
            data['from_node'].append(route[2][1])
            data['to_node'].append(route[3][1])
            data['from_id'].append(bb.get_bb_id(route[2][2:4]))
            data['to_id'].append(bb.get_bb_id(route[3][2:4]))
            data['transport'].append(route[0])
            data['from_airport'].append(route[2][0])
            data['to_airport'].append(route[3][0])
            data['from_airport_id'].append(ap.get_airport_id(route[2][0]))
            data['to_airport_id'].append(ap.get_airport_id(route[3][0]))
            data['price'].append(route[11][0][0])
            data['currency'].append(route[11][0][1])
            data['price_local'].append('')
            data['currency_local'].append('')
            data['distance_km'].append('')
            data['duration_min'].append(int(route[4] / 60)) # sec to min
        else:
            data['path_id'].append(path_num)
            data['path_name'].append(path[4])
            data['from_node'].append(route[6][1])
            data['to_node'].append(route[7][1])
            data['from_id'].append(bb.get_bb_id(route[6][2:4]))
            data['to_id'].append(bb.get_bb_id(route[7][2:4]))
            data['transport'].append(route[1])
            data['from_airport'].append('')
            data['to_airport'].append('')
            data['from_airport_id'].append('')
            data['to_airport_id'].append('')
            data['price'].append(route[13][0][0])
            data['currency'].append(route[13][0][1])
            data['price_local'].append(route[14][0][0])
            data['currency_local'].append(route[14][0][1])
            data['distance_km'].append(int(route[5]))
            data['duration_min'].append(int(route[3] / 60)) # sec to min
                            
tmp_df = pd.DataFrame(data)

# uncomment line below to make output in folder "../files/output/csv_output/"
tmp_df.to_csv(output_file) 

print(tmp_df)

    path_id                                    path_name  \
0         0         Fly Berlin Brandenburg to Montevideo   
1         0         Fly Berlin Brandenburg to Montevideo   
2         0         Fly Berlin Brandenburg to Montevideo   
3         1          Train to Leipzig, fly to Montevideo   
4         1          Train to Leipzig, fly to Montevideo   
5         1          Train to Leipzig, fly to Montevideo   
6         1          Train to Leipzig, fly to Montevideo   
7         2          Train to Hamburg, fly to Montevideo   
8         2          Train to Hamburg, fly to Montevideo   
9         2          Train to Hamburg, fly to Montevideo   
10        2          Train to Hamburg, fly to Montevideo   
11        3  Fly Berlin Brandenburg to Ezeiza, car ferry   
12        3  Fly Berlin Brandenburg to Ezeiza, car ferry   
13        3  Fly Berlin Brandenburg to Ezeiza, car ferry   
14        3  Fly Berlin Brandenburg to Ezeiza, car ferry   
15        4    Train, train to Prague, f